In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import glob

In [ ]:
folder = '../workflow_results/Simulation_Files'

In [ ]:
list_files = glob.glob(folder+"/**/**/**/report.csv")
list_files = pd.DataFrame(list_files)
list_bldg = list_files.iloc[:,0].str.split(folder, expand=True).iloc[:,1].str.split("Baseline",expand=True).iloc[:,0].str.replace("\\","").tolist()
list_bldg

In [ ]:
dict_monthgroup = {
    'Summer':[6, 7, 8],
    'Winter':[12, 1, 2],
    'Shoulder':[3, 4, 5, 9, 10, 11]
}

In [ ]:
list_ov = ['Electricity', 'Natural Gas']

In [ ]:
datestart = '2019-01-01 00:00:00'
dateend = '2019-12-31 23:00:00'

df_combined = pd.DataFrame([])
df_combined['reading_time'] = pd.date_range(datestart, dateend, freq='60T')
df_combined = df_combined.set_index(['reading_time'])

In [ ]:
for energy in list_ov:

    for bldg in list_bldg:

        print("processing bldg {}".format(bldg))

        list_files = glob.glob(folder + '/' + bldg + '/' + "/Baseline/*HourlyConsumptionByFuelToCSV/report.csv")        
        df = pd.read_csv(list_files[0])
        df_combined[str(bldg)+"_"+energy] = df[energy].values


In [ ]:
df_combined

In [ ]:
for energy in list_ov:
    
    fig = make_subplots(
        rows=len(list_ov), 
        cols=len(dict_monthgroup), 
        vertical_spacing=0.1,
        shared_yaxes=True
    )

    # select from https://plotly.com/python/discrete-color/
    colorscale = px.colors.qualitative.Dark2
    count_color = 0
        
    filetolook = 'report.csv'

    df_temp = df_combined.loc[:, df_combined.columns.str.contains(energy)].copy()
    df_temp = df_temp.sum(axis=1).to_frame()
    df_temp.columns = [energy] # unit in Joules
    df_temp[energy] = df_temp[energy] * 2.77778e-7 # converting to kWh (or kW if hourly data)

    for idx, daytype in enumerate(['Weekday','Weekend']):

        if daytype=='Weekday':
            list_value = [0,1,2,3,4]
        elif daytype=='Weekend':
            list_value = [5,6]

        df_daytype = df_temp.loc[df_temp.index.weekday.isin(list_value), :].copy()

        pos_ssn = 1
        for ssn in dict_monthgroup:

            df_ssn = df_daytype.loc[df_daytype.index.month.isin(dict_monthgroup[ssn]),:].copy()
            df_ssn.loc[:, 'HourOfDay'] = df_ssn.index.hour
            df_ssn.loc[:, 'Date'] = df_ssn.index.date
            df_ssn = df_ssn.pivot_table(index='HourOfDay',columns='Date',values=energy)

            # average
            fig.add_trace(go.Scatter(
                x=df_ssn.index,
                y=df_ssn.mean(axis=1),
                name=energy,
                opacity=0.7,
                line = dict(
                    color=colorscale[count_color], 
                    width=2, 
                ),
                legendgroup=energy,
                showlegend=False
            ),row=idx+1, col=pos_ssn)

            # upperbound
            fig.add_trace(go.Scatter(
                x=df_ssn.index,
                y=df_ssn.max(axis=1),
                name=energy,
                opacity=0.7,
                line = dict(
                    width=0, 
                ),
                legendgroup=energy,
                showlegend=False
            ),row=idx+1, col=pos_ssn)

            # lowerbound
            fig.add_trace(go.Scatter(
                x=df_ssn.index,
                y=df_ssn.min(axis=1),
                name=energy,
                opacity=0.7,
                line = dict(
                    width=0, 
                ),
                fillcolor='rgba(68, 68, 68, 0.3)',
                fill='tonexty',
                legendgroup=energy,
                showlegend=False
            ),row=idx+1, col=pos_ssn)

            fig.update_yaxes(
                title=energy + " [kW]" + f"<br><sub>{daytype}</sub>",
                row=idx+1,
                col=1
            )
            
            fig.update_xaxes(
                title=ssn,
                row=len(list_ov),
                col=pos_ssn,
            )

            pos_ssn+=1

    fig.update_layout(
        title="Daily load profiles",
        width=900,
        height=350,
        margin=dict(
            l=100,
            r=50,
            t=50,
            b=50
        ),
        plot_bgcolor='rgb(242,242,242)',
    )
    
    fig.show()